In [1]:
import spacy
import csv
from collections import namedtuple
from spacy.matcher import PhraseMatcher
import re

In [2]:
o = ""

for page in range(1,60):
    with open('edited_from_ocr/Complaint-p{}-normal.txt'.format(str(page)),'r') as output:
        o = "\n".join([o, output.read()])

In [3]:
o = o.replace("\n", " ").replace("Case 1:18-cv-03501 Document 1 Filed 04/20/18 ", "")\
    .replace("$$", "§§").replace(" $ ", " § ").replace(".$ ", ". § ").replace("С", "C")\
    .replace(" S ", " § ").replace(" SS", " §§").replace(" 55 "," §§ ").replace("US.C.", "U.S.C.")\
    .replace("U.S.C. 8 ", "U.S.C. § ").replace("&well","and well").replace(" & ",", ")

In [4]:
for n in range(66):
    o = o.replace(f"Page {n} of 66", f'**{n}**')
    # o = o.replace(f"Page {n} of 66", f'<page number="{n}" />')

In [5]:
filename = "./edited_from_ocr/full_complaint.txt"

with open(filename, 'w') as fout:
    fout.write(o)

In [6]:
# Spacy parsing probably needs to be done before tags are added.

o = "<complaint>" + o + "</complaint>"

In [7]:
coa = re.compile(r"((COUNT [IVX]+)[^/d]+?\)[^/d]+?\))")

In [8]:
complaint = o[:10000] + re.sub(coa, r'</cause_of_action><cause_of_action count="\2"><heading>\1</heading>', o[10000:])

In [9]:
complaint = complaint.replace("</cause_of_action>", "", 1)\
                     .replace("PRAYER FOR RELIEF", "</cause_of_action>PRAYER FOR RELIEF")

In [10]:
import xml.etree.ElementTree as ET
root = ET.fromstring(complaint)

In [11]:
def tag_allegations(s: str):
    s = re.sub(r" (\d\d\d)\. ", r"""<allegation para="\1">""", s, count=1)
    s = re.sub(r" (\d\d\d)\. ", r"""</allegation><allegation para="\1">""", s)
    return s + "</allegation>"

# o = tag_allegations(o, 146, 153)

In [12]:
for child in root.getchildren():
    child[0].tail = tag_allegations(child[0].tail)

In [13]:
c = ET.tostring(root, encoding="unicode", method="html")

In [14]:
d = c.replace("&lt;", "<").replace("&gt;", ">")

In [15]:
with open("./xml/complaint.xml", 'w') as fout:
    fout.write(d)

In [ ]:
for a in root.getchildren():
    for b in a.getchildren():
        print(b)

In [ ]:
parties = []

with open("./data/parties.csv", newline="") as infile:
    reader = csv.reader(infile)
    Party = namedtuple("Party", next(reader))  # get names from column headers
    for data in map(Party._make, reader):
        parties.append(data)
    

In [ ]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(o)

In [ ]:
# This is probably overkill. I should just use string methods to highlight/tag the names.

party_matches = []

def on_match(matcher, doc, uid, matches):
    print(f'{uid} matched!')
    # party_matches.append((uid, matches, "Party"))

matcher = PhraseMatcher(nlp.vocab)
for p in parties:
    if len(p.name) < 10:
        matcher.add(p.short_name, on_match, nlp(p.short_name), nlp(p.name))
    else: matcher.add(p.short_name, on_match, nlp(p.short_name))
    
# matches = matcher(doc)

In [ ]:
Entity = namedtuple("Entity", "text start_char end_char label_")

In [ ]:
entities = [Entity(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents
            if e.label_ not in ("CARDINAL", "ORDINAL", "DATE", "MONEY", "TIME", "LAW", "FAC")]

In [ ]:
for sent in list(doc.sents)[460:470]:
    print(sent.text)